In [ ]:
# import the necessary packages
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2

In [ ]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True,
	help="path to facial landmark predictor")
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
args = {
	"shape_predictor":"models/shape_predictor_68_face_landmarks.dat",
	"image":"/home/sidhant/Projects/drowsiness-detection-and-alert/assets/02_face.jpg"
}

"""
Change:
args = {
	"shape_predictor":"models/shape_predictor_68_face_landmarks.dat",
	"image":"assets/02_face.jpg"
}

to:
args = vars(ap.parse_args())

in order to get the location from the command line
"""